In [28]:
import requests
import os
import random
from random import randrange
import re
import json

In [29]:
#fonction pour écrire dans un fichier json
def write_json(chemin,contenu):
    with open(chemin,"w",encoding="utf-8") as w :
        w.write(json.dumps(contenu, indent=2, ensure_ascii=False))

In [30]:
#fonction pour récupérer le contenu d'un fichier json
def ouvrir_json(chemin) :
    with open(chemin,encoding="utf-8") as f :
        contenu = json.load(f)
    return contenu

In [31]:
#https://github.com/athalhammer/danker
#enregistrement des identifiants Wikidata et de leur score dans un dictionnaire
#le fichier 2021-11-15.allwiki.links.rank étant trop lourd, il n'est pas dans le répertoire ressources mais est téléchargeable au lien :
#https://danker.s3.amazonaws.com/index.html
def dico_wikidata_ranks(chemin="ressources/2021-11-15.allwiki.links.rank") :
    dico_ranks = {}
    cpt = 0
    with open(chemin,"r",encoding="utf-8") as f :
        ligne = f.readline()
        ligne = ligne[:len(ligne)-1]

        while ligne :
            cpt+=1
            liste = ligne.split("\t")
            dico_ranks[liste[0]] = float(liste[1])

            ligne = f.readline()
            ligne = ligne[:len(ligne)-1]

    return dico_ranks

In [32]:
def statistiques_rangs(dico) :
    longueur = len(dico)
    
    cinquieme = int(longueur/5)
    dico_trie = {}
    #tri du dictionnaire
    #https://waytolearnx.com/2019/04/comment-trier-un-dictionnaire-par-cle-ou-par-valeur-en-python.html#:~:text=Si%20nous%20voulons%20trier%20les,par%20ordre%20croissant%20par%20d%C3%A9faut).
    for k, v in sorted(dico.items(), key=lambda x: x[1]):
        dico_trie[k] = v
    #classement en niveaux des lexèmes
    dico_niveaux = {}
    
    liste_lexemes = list(dico_trie.keys())
    
    dico_niveaux['D'] = liste_lexemes[2*cinquieme:3*cinquieme]
    dico_niveaux['M'] = liste_lexemes[3*cinquieme:4*cinquieme]
    dico_niveaux['F'] = liste_lexemes[4*cinquieme:]
 
    
    return dico_niveaux

In [44]:
def ranking_properties(dico_ranks,dico_reponses) :
    notes = {}
    cpt = 0
    #classement des proprietes de la reponse
    for k,v in dico_reponses.items() :
        #si la valeur (de la propriété) est dans le dictionnaire de rangs
        if dico_ranks.get(k) != None :
            rang = dico_ranks[k]
            notes[rang] = k
            cpt+=1
   
    #sortie : {score : identifiant}
    notes_triees = sorted(notes.keys(),reverse=True)
    dico_trie = {}
    for n in notes_triees :
        dico_trie[n] = notes[n]
        
    return dico_trie

In [33]:
def requete_tableaux(lang) :
    nom = "requete_tableaux_"+lang+".json"
    if os.path.exists("ressources/"+nom) :
        return nom
    estValide = False
    while estValide != True :
        requete= """
        SELECT ?reponse ?reponseLabel WHERE
        {
          ?reponse wdt:P1343 wd:Q66362718;
                   rdfs:label ?reponseLabel.
          FILTER(lang(?reponseLabel)="%s")
        }
        """%(lang)
        url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
        response = requests.get(url, params={'query': requete, 'format': 'json'})
        if response.status_code == 200 :
            estValide = True
            data = response.json()
            write_json(nom,data)
    
    return nom

In [34]:
def requete_capitales(lang) :
    nom = "requete_capitales_"+lang+".json"
    if os.path.exists("ressources/"+nom) :
        return nom
    estValide = False
    while estValide != True :
        requete= """
        SELECT ?reponse ?reponseLabel WHERE
        {
          ?reponse wdt:P31 wd:Q5119.
          ?pays wdt:P36 ?reponse.
          ?pays wdt:P31 wd:Q6256.
          ?reponse rdfs:label ?reponseLabel.
          
          FILTER(lang(?reponseLabel) = '%s')
        }
        """%(lang)
        url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
        response = requests.get(url, params={'query': requete, 'format': 'json'})
        if response.status_code == 200 :
            estValide = True
            data = response.json()
            write_json(nom,data)
    
    return nom

In [35]:
def requete_actrices(lang) :
    nom = "requete_actrices_"+lang+".json"
    if os.path.exists("ressources/"+nom) :
        return nom
    estValide = False
    while estValide != True :
        requete= """
       SELECT ?reponse ?reponseLabel WHERE
        {
          ?reponse wdt:P31 wd:Q5.
          ?reponse wdt:P166 wd:Q103618.
          ?reponse rdfs:label ?reponseLabel.
          FILTER(lang(?reponseLabel)="%s")
        }
        """%(lang)
        url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
        response = requests.get(url, params={'query': requete, 'format': 'json'})
        if response.status_code == 200 :
            estValide = True
            data = response.json()
            write_json(nom,data)
    
    return nom

In [36]:
def requete_fleuves(lang) :
    nom = "requete_fleuves_"+lang+".json"
    if os.path.exists("ressources/"+nom) :
        return nom
    estValide = False
    while estValide != True :
        requete= """
        SELECT ?reponse ?reponseLabel WHERE
        {
          {
            ?reponse wdt:P31 wd:Q4022;
                   wdt:P30 wd:Q49;
                   rdfs:label ?reponseLabel.
            FILTER(lang(?reponseLabel)="%s").
          }
          UNION
          {
            ?reponse wdt:P31 wd:Q4022;
                   wdt:P30 wd:Q18;
                   rdfs:label ?reponseLabel.
            FILTER(lang(?reponseLabel)="%s").
           }
        }
        """%(lang,lang)
        url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
        response = requests.get(url, params={'query': requete, 'format': 'json'})
        if response.status_code == 200 :
            estValide = True
            data = response.json()
            write_json(nom,data)
    
    return nom

In [37]:
def structuration_requete(fichier) :

    resultats_json = ouvrir_json(fichier)
    liste_niv1 = resultats_json["results"]["bindings"]
    
    resultats = {}

    for un in liste_niv1 :
        if un['reponse']['value'] not in resultats :
            k = un['reponse']['value'].split("http://www.wikidata.org/entity/")[1]
            resultats[k] = un['reponseLabel']['value']
  
        
    return resultats

In [38]:
#dico_rangs -> dico_niveaux
def choix_reponse(lang,dico_reponses,dico_niveaux,diff) :
    
    cpt = 0
    
    estValide = False
    while estValide != True :
    
        rep_poss = random.choice(list(dico_reponses.keys()))
       
        while rep_poss not in dico_niveaux[diff] :
            #on veut éviter une boucle infinie, on limite ainsi les tentatives à 10
            if cpt > 20 :
                break
            #rep_poss est un identifiant
            rep_poss = random.choice(list(dico_reponses.keys()))
            cpt+=1
        #reponse est une chaine
        reponse = dico_reponses[rep_poss]

        requete = """
        SELECT ?proprieteLabel ?valeurProp ?valeurPropLabel {
          BIND (wd:%s AS ?entite)
          ?entite ?p ?o .
          ?o ?prop ?valeurProp .
          ?propriete wikibase:claim ?p.
          ?propriete wikibase:statementProperty ?prop.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "%s" }
        }
        """%(rep_poss,lang)
        url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
        response = requests.get(url, params={'query': requete, 'format': 'json'})
        if response.status_code != 200 :
            pass
        else :
            estValide = True
            data = response.json()
            dico_reponse = {}
            liste_niv1 = data["results"]["bindings"]
            for l in liste_niv1 :
                if re.search(r'^http://www.wikidata.org/entity/.*',l['valeurProp']['value']) :
                    identifiant = l['valeurProp']['value'].split("http://www.wikidata.org/entity/")[1]
                    if identifiant not in dico_reponse :
                        dico_reponse[identifiant] = (l['proprieteLabel']['value'],l['valeurPropLabel']['value'])
                else :
                    identifiant = l['proprieteLabel']['value']
                    if identifiant not in dico_reponse :
                        dico_reponse[identifiant] = (identifiant,l['valeurPropLabel']['value'])
    return dico_reponse,reponse

In [39]:
#ajout du theme
def preparation_reponse(dico_reponse,dico_rangs,diff,lang,reponse,theme) :
    classement_reponse = ranking_properties(dico_rangs,dico_reponse)
    
    indices = []
    indices_fin = []
    
    #PEINTURE
    if theme.lower() == 'p' :
        blacklist = ["statut des droits d'auteur","copyright status"]

        for k,v in classement_reponse.items() :
            #on enleve les proprietes non pertinents et on verifie que la reponse entiere n'est pas dans les indices
            if dico_reponse[v][0] not in blacklist and dico_reponse[v][1] != reponse :
                indices.append(dico_reponse[v])

        #j'ajoute des indices utiles
        if lang == "fr" :
            #on verifie que cette cle existe bien
            if dico_reponse.get('date de fondation ou de création') != None :
                indices.insert(0,(dico_reponse['date de fondation ou de création'][0],dico_reponse['date de fondation ou de création'][1].split("-")[0]))
        if lang == "en" :
            if dico_reponse.get('inception') != None :
                indices.insert(0,(dico_reponse['inception'][0],dico_reponse['inception'][1].split("-")[0]))

    #CAPITALES
    if theme.lower() == 'c' :
        blacklist = ["instance of","twinned administrative body","capital of","described by source","capitale de","nature de l'élément","jumelage ou partenariat","décrit par","à ne pas confondre avec","projet Wikimédia s’intéressant à l'élément"]
        for k,v in classement_reponse.items() :
            #on enleve les proprietes non pertinents et on verifie que la reponse entiere n'est pas dans les indices
            if dico_reponse[v][0] not in blacklist and dico_reponse[v][1] != reponse :
                #on vérifie que les indices n'ont pas la réponse dans la question
                estCategorie = re.match(rf'.*?{reponse}.*',dico_reponse[v][1])
                #on vérifie que les indices ont bien un label (et pas uniquement un identifiant qui s'affiche)
                sansLabel = re.match(r'Q\d+',dico_reponse[v][1])
                if not estCategorie and not sansLabel :
                    indices.append(dico_reponse[v])
                    
        if dico_reponse.get('population') != None :
            indices.insert(0,dico_reponse['population'])
        if lang == "fr" :
            if dico_reponse.get('indicatif téléphonique local') != None :
                    indices.insert(1,dico_reponse['indicatif téléphonique local'])
            if dico_reponse.get('superficie') != None :
                    indices.insert(2,dico_reponse['superficie'])
        if lang == "en" :
            if dico_reponse.get('local dialing code') != None :
                    indices.insert(1,dico_reponse['local dialing code']) 
            if dico_reponse.get('area') != None :
                    indices.insert(2,dico_reponse['area'])
    
    #ACTRICES 
    if theme.lower() == "a" :
        blacklist = ["on focus list of Wikimedia project","projet Wikimédia s’intéressant à l'élément","décrit par","described by source","instance of","nature de l'élément","sex or gender","sexe ou genre","name in native language","nom dans la langue maternelle de la personne","given name","prénom","family name","nom de famille"]
        for k,v in classement_reponse.items() :
            estExclu = re.match(rf'.*?{reponse}.*',dico_reponse[v][1])
            sansLabel = re.match(r'Q\d+',dico_reponse[v][1])
            #on vérifie que les indices ont bien un label (et pas uniquement un identifiant qui s'affiche)
            if dico_reponse[v][0] not in blacklist and not estExclu and not sansLabel :
                #eliminer la propriété vainqueure oscar parce que le thème l'indique déjà
                if dico_reponse[v][1] != "Oscar de la meilleure actrice" and dico_reponse[v][1] != "Academy Award for Best Actress" :
                    indices.append(dico_reponse[v])
                    
        if lang == "fr" :
            if dico_reponse.get('date de naissance') != None :
                indices.insert(0,(dico_reponse['date de naissance'][0],dico_reponse['date de naissance'][1].split('T')[0]))
            if dico_reponse.get('date de mort') != None :
                indices.insert(1,(dico_reponse['date de mort'][0],dico_reponse['date de mort'][1].split('T')[0]))
        if lang == "en" :
            if dico_reponse.get('date of birth') != None :
                indices.insert(0,(dico_reponse['date of birth'][0],dico_reponse['date of birth'][1].split('T')[0]))
            if dico_reponse.get('date of death') != None :
                indices.insert(1,(dico_reponse['date of death'][0],dico_reponse['date of death'][1].split('T')[0]))
    #FLEUVES
    if theme.lower() == "f" :
        blacklist = ["nature de l'élément","instance of","native label","nom dans la langue originelle","described by source","décrit par","on focus list of Wikimedia project","projet Wikimédia s’intéressant à l'élément"]
        for k,v in classement_reponse.items() :
            estExclu = re.match(rf'.*?{reponse}.*',dico_reponse[v][1])
            sansLabel = re.match(r'Q\d+',dico_reponse[v][1])
            #on vérifie que les indices ont bien un label (et pas uniquement un identifiant qui s'affiche)
            if dico_reponse[v][0] not in blacklist and not estExclu and not sansLabel :
                indices.append(dico_reponse[v])
        if lang == "fr" :
            if dico_reponse.get('longueur') != None :
                indices.insert(0,(dico_reponse['longueur'][0],dico_reponse['longueur'][1]+" km"))
        if lang == "en" :
            if dico_reponse.get('length') != None :
                indices.insert(0,(dico_reponse['length'][0],dico_reponse['length'][1]+" mile")) 
                               
                
    #print(indices)
    if diff.lower() == 'f' :
        indices_fin = indices[:10]

    if diff.lower() == 'm' :
        indices_fin = indices[2:12]

    if diff.lower() == 'd' :
        indices_fin = indices[4:14]
    
    #on gère les cas où il n'y a pas d'indice
    if len(indices_fin) < 3 :
        indices_fin = indices[:10]
    
    return indices_fin

In [25]:
dico_ranks = dico_wikidata_ranks()
dico_niveaux = statistiques_rangs(dico_ranks)

In [42]:
def jeu(dico_ranks,dico_niveaux) :
    
    cpt = 0
    print("Bienvenue ! Nous allons vous proposer une liste d'indices et vous devez trouver la réponse !")
    print("Chargement...")
    
    langue = input("Souhaitez vous effectuer ce jeu en français (fr) ou en anglais (en) ?")
    diff = input("Niveau de difficulté : Facile (F) ? Moyen (M) ? Difficile (D) ?")
    
    theme = input("Quel thème choisissez-vous ? Peintures célèbres (P), Capitales du monde (C), Actrices oscarisées (A), Fleuves d'Amérique du Nord et d'Amérique du Sud (F) ?")
    
    if theme.lower() == 'p' :
        nom_fichier = requete_tableaux(langue)
    if theme.lower() == 'c' :
        nom_fichier = requete_capitales(langue)
    if theme.lower() == 'a' :
        nom_fichier = requete_actrices(langue)
    if theme.lower() == 'f' :
        nom_fichier = requete_fleuves(langue)
        
    
    all_reponses = structuration_requete("ressources/"+nom_fichier)
    while cpt < 5 :
        dico_reponse,reponse = choix_reponse(langue,all_reponses,dico_niveaux,diff)
        indices = preparation_reponse(dico_reponse,dico_ranks,diff,langue,reponse,theme)
        print("-"*60)
        print("Voici les indices : ")
        for i in indices :
            print("%s : %s"%(i[0],i[1]))

        proposition = input("Quelle est votre réponse ?") 

        if proposition.lower() == reponse.lower() :
            print("Bonne réponse !")
        else :
            print("Non ! La bonne réponse est : %s "%(reponse))
        print("-"*60)
        cpt+=1
    print("Au revoir !")
    return 

In [ ]:
jeu(dico_ranks,dico_niveaux)